In [1]:
# Import dependencies
import pandas as pd
import unirest
import pprint, json
import time, os
from config import api_key

In [2]:
# Create a DataFrame with all the Players IDs
player = pd.DataFrame({"playerId":[x for x in range(1, 2241, 1)]})
player.head()

,playerId
0,1
1,2
2,3
3,4
4,5


In [3]:
# Add new fields to DF.
player['firstName'] = None 
player['lastName'] = None 
player['fullname'] = None 
player['affiliation'] = '' 
player['affiliation_city'] = '' 
player['affiliation_country'] = '' 
player['collegeName'] = ''
player['country'] = '' 
player['dateOfBirth'] = None
player['heightInMeters'] = None 
player['startNba'] = None 
player['teamId'] = None 
player['weightInKilograms'] = None
player['yearsPro'] = None

count = 0
url = "https://api-nba-v1.p.rapidapi.com/players/playerId/"
for index, row in player.iterrows(): 
    # Sleep for one second between each request.
    time.sleep(1)
    count += 1;
    if count >= 1:
        break
    
    curr_url = f"{url}{row['playerId']}" 

    # Execute the current request
    response = unirest.get(curr_url,
      headers={
        "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com",
        "X-RapidAPI-Key": api_key,
      }
    )
    r = response.body
    affiliation  = r['api']['players'][0]['affiliation'] 
    
    # Fill new fields
    player.loc[index, 'firstName'] =  r['api']['players'][0]['firstName'] 
    player.loc[index, 'lastName'] =  r['api']['players'][0]['lastName'] 
    player.loc[index, 'fullname'] = r['api']['players'][0]['firstName'] + ' ' + r['api']['players'][0]['lastName']
    player.loc[index, 'affiliation'] = affiliation
    player.loc[index, 'collegeName'] = r['api']['players'][0]['collegeName']
    player.loc[index, 'country'] =   r['api']['players'][0]['country']
    player.loc[index, 'dateOfBirth'] = r['api']['players'][0]['dateOfBirth']
    player.loc[index, 'heightInMeters'] =  r['api']['players'][0]['heightInMeters'] 
    player.loc[index, 'startNba'] =  r['api']['players'][0]['startNba'] 
    player.loc[index, 'teamId'] =  r['api']['players'][0]['teamId'] 
    player.loc[index, 'weightInKilograms'] = r['api']['players'][0]['weightInKilograms'] 
    player.loc[index, 'yearsPro'] =  r['api']['players'][0]['yearsPro']

    # Split the affiliation if is filled 
    if len(affiliation.split("/")) == 2:
        player.loc[index, 'affiliation_city'] = affiliation.split("/")[0] 
        player.loc[index, 'affiliation_country'] = affiliation.split("/")[1]   


In [4]:
# Assign the players' file to a variable 
players_file = os.path.join(".", "Resources", "players.csv")
# player.to_csv(players_file, encoding="utf-8", index=False)

In [5]:
# Starting cleasing. Checking for duplicated players.
players = pd.read_csv(players_file)

# Drop useless fields
players.drop(axis=1, columns=["playerId", 'firstName','affiliation','lastName','dateOfBirth','heightInMeters','startNba','teamId','weightInKilograms','yearsPro', 'collegeName','country'],inplace=True) 

# Drop Nas 
players = players.dropna(how="any")

# Drop duplicated
players.drop_duplicates('fullname', 'last', inplace=True)


In [6]:
# Merge the files
statistics_file = os.path.join(".", "Resources", "perplayerstatistics.csv")
statistics = pd.read_csv(statistics_file)
merged = statistics.merge(players.rename(columns={'fullname':'name'}),how='left')
len(merged)

merged_file = os.path.join(".", "Resources", "final_file.csv")
merged.to_csv(merged_file, encoding="utf-8", index=False)